## American Consumer Expenditures 2023

In [ ]:
from dash import Dash, dcc, html, Input, Output, State
import plotly.graph_objects as go
import plotly.io as pio

app = Dash(__name__)

# Layout for the app
app.layout = html.Div([
    html.H4('Detailed Sankey Diagram: Average American Expenditure (2023)'),
    dcc.Graph(id="graph"),
    html.P("Opacity"),
    dcc.Slider(id='slider', min=0, max=1, value=0.8, step=0.1),
    html.Button("Save as PDF", id="save-pdf", n_clicks=0)
])

# Callback for creating the Sankey diagram
@app.callback(
    Output("graph", "figure"),
    Input("slider", "value")
)
def display_sankey(opacity):
    categories = [
        "Housing", "Transportation", "Food", "Personal Insurance and Pensions",
        "Healthcare", "Entertainment", "Education", "Miscellaneous", "Other"
    ]
    subcategories = {
        "Housing": ["Owned dwellings", "Rented dwellings", "Other lodging", "Other housing expenses"],
        "Transportation": ["Vehicle purchases", "Gasoline", "Public/other transportation"],
        "Food": ["Food at home", "Food away from home"],
        "Personal Insurance and Pensions": ["Life insurance", "Pensions/Social Security"],
        "Healthcare": ["Health insurance", "Medical services"],
        "Entertainment": ["Fees/admissions", "Pets, toys, hobbies", "Other entertainment"],
        "Education": ["Tuition", "Test prep/tutoring", "School supplies"],
        "Miscellaneous": ["Legal fees", "Other misc."],
        "Other": []  # No subcategories for "Other"
    }

    percentages = {
        "Total Income": 100,
        "Housing": 32.9,
        "Owned dwellings": 12.5,
        "Rented dwellings": 10.5,
        "Other lodging": 2.0,
        "Other housing expenses": 7.9,  # Added missing percentage
        "Transportation": 17.0,
        "Vehicle purchases": 8.0,
        "Gasoline": 3.2,
        "Public/other transportation": 5.8,
        "Food": 12.9,
        "Food at home": 6.0,
        "Food away from home": 6.9,
        "Personal Insurance and Pensions": 12.4,
        "Life insurance": 2.0,
        "Pensions/Social Security": 10.4,
        "Healthcare": 8.0,
        "Health insurance": 6.0,
        "Medical services": 2.0,
        "Entertainment": 4.7,
        "Fees/admissions": 1.5,
        "Pets, toys, hobbies": 2.0,
        "Other entertainment": 1.2,
        "Education": 2.1,
        "Tuition": 1.0,
        "Test prep/tutoring": 0.5,
        "School supplies": 0.6,
        "Miscellaneous": 1.5,
        "Legal fees": 0.8,
        "Other misc.": 0.7,
        "Other": 5.8
    }

    labels = ["Total Income"] + [
        f"{key} ({value}%)" for key, value in percentages.items() if key != "Total Income"
    ]
    sources = []
    targets = []
    values = []

    for category in categories:
        sources.append(0)
        targets.append(labels.index(f"{category} ({percentages[category]}%)"))
        values.append(percentages[category])

        if category in subcategories:
            for subcat in subcategories[category]:
                sources.append(labels.index(f"{category} ({percentages[category]}%)"))
                targets.append(labels.index(f"{subcat} ({percentages[subcat]}%)"))
                values.append(percentages[subcat])

    main_colors = {
        "Housing": f'rgba(0, 0, 139, {opacity})',
        "Transportation": f'rgba(169, 169, 169, {opacity})',
        "Food": f'rgba(34, 139, 34, {opacity})',
        "Personal Insurance and Pensions": f'rgba(100, 149, 237, {opacity})',
        "Healthcare": f'rgba(220, 20, 60, {opacity})',
        "Entertainment": f'rgba(255, 165, 0, {opacity})',
        "Education": f'rgba(123, 104, 238, {opacity})',
        "Miscellaneous": f'rgba(47, 79, 79, {opacity})',
        "Other": f'rgba(255, 255, 255, {opacity})'
    }

    node_colors = []
    for label in labels:
        if "Total Income" in label:
            node_colors.append(f'rgba(235, 235, 235, {opacity})')
        elif any(cat in label for cat in main_colors.keys()):
            category = next(cat for cat in main_colors.keys() if cat in label)
            node_colors.append(main_colors[category])
        else:
            parent_category = next(
                (parent for parent, subs in subcategories.items() if any(sub in label for sub in subs)), None
            )
            if parent_category:
                lighter_color = main_colors[parent_category].replace("0.8", "0.4")
                node_colors.append(lighter_color)
            else:
                node_colors.append(f'rgba(0, 51, 102, {opacity})')

    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=15,
            line=dict(color="black", width=0.5),
            label=labels,
            color=node_colors
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=[node_colors[s] for s in sources]
        )
    ))

    fig.update_layout(
        title_text="Average American Expenditure (2023)",
        font=dict(size=8),
        height=1200,
        margin=dict(l=10, r=10, t=50, b=10)
    )
    return fig

# Callback for saving as PDF
@app.callback(
    Output("save-pdf", "n_clicks"),
    Input("save-pdf", "n_clicks"),
    State("graph", "figure")
)
def save_pdf(n_clicks, figure):
    if n_clicks > 0:
        fig = go.Figure(figure)
        pio.write_image(fig, "sankey_diagram.pdf", format="pdf")
    return n_clicks

if __name__ == '__main__':
    app.run_server(debug=True)


In [58]:
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

app = Dash(__name__)

# Layout for the app
app.layout = html.Div([
    html.H4('Sankey Diagram: Percent Distribution of Expenditures (2023)'),
    dcc.Graph(id="graph"),
    html.P("Opacity"),
    dcc.Slider(id='slider', min=0, max=1, value=0.8, step=0.1),
    html.Button("Save as PDF", id="save-button", n_clicks=0),
    html.Div(id="save-output")  # To display save confirmation
])

# Callback for generating and saving the Sankey diagram
@app.callback(
    [Output("graph", "figure"),
     Output("save-output", "children")],
    [Input("slider", "value"),
     Input("save-button", "n_clicks")]
)
def display_sankey(opacity, n_clicks):
    # Data from Table B for 2023
    categories = [
        "Food", "Alcoholic beverages", "Housing", "Apparel and services",
        "Transportation", "Healthcare", "Entertainment", "Personal care products and services",
        "Reading", "Education", "Tobacco products and smoking supplies",
        "Miscellaneous", "Cash contributions", "Personal insurance and pensions"
    ]

    percentages = {
        "Food": 12.9,
        "Alcoholic beverages": 0.8,
        "Housing": 32.9,
        "Apparel and services": 2.6,
        "Transportation": 17.0,
        "Healthcare": 8.0,
        "Entertainment": 4.7,
        "Personal care products and services": 1.2,
        "Reading": 0.2,
        "Education": 2.1,
        "Tobacco products and smoking supplies": 0.5,
        "Miscellaneous": 1.5,
        "Cash contributions": 3.1,
        "Personal insurance and pensions": 12.4
    }

    # Create nodes and links
    labels = ["Total Expenditures"] + [f"{category} ({percent}%)" for category, percent in percentages.items()]
    sources = []
    targets = []
    values = []

    # Link Total Expenditures to all categories
    for category, percent in percentages.items():
        sources.append(0)  # Index of "Total Expenditures"
        targets.append(labels.index(f"{category} ({percent}%)"))
        values.append(percent)

    # Define colors for categories
    main_colors = {
        "Food": f'rgba(34, 139, 34, {opacity})',
        "Alcoholic beverages": f'rgba(160, 82, 45, {opacity})',
        "Housing": f'rgba(70, 130, 180, {opacity})',
        "Apparel and services": f'rgba(255, 99, 71, {opacity})',
        "Transportation": f'rgba(169, 169, 169, {opacity})',
        "Healthcare": f'rgba(220, 20, 60, {opacity})',
        "Entertainment": f'rgba(255, 165, 0, {opacity})',
        "Personal care products and services": f'rgba(123, 104, 238, {opacity})',
        "Reading": f'rgba(64, 224, 208, {opacity})',
        "Education": f'rgba(100, 149, 237, {opacity})',
        "Tobacco products and smoking supplies": f'rgba(47, 79, 79, {opacity})',
        "Miscellaneous": f'rgba(211, 211, 211, {opacity})',
        "Cash contributions": f'rgba(218, 165, 32, {opacity})',
        "Personal insurance and pensions": f'rgba(0, 51, 102, {opacity})'
    }

    node_colors = [f'rgba(0, 51, 102, {opacity})'] + [main_colors[cat] for cat in percentages.keys()]

    # Create Sankey diagram
    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=15,
            line=dict(color="black", width=0.5),
            label=labels,  # Include percentages directly in the labels
            color=node_colors
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=[main_colors[cat] for cat in percentages.keys()]
        )
    ))

    fig.update_layout(
        title_text="Sankey Diagram: Percent Distribution of Expenditures (2023)",
        font_size=10,
        height=800,
        margin=dict(l=10, r=10, t=50, b=10)
    )

    # Save as PDF if the button is clicked
    if n_clicks > 0:
        pdf_path = "Sankey_Diagram_Expenditures_2023.pdf"
        fig.write_image(pdf_path)
        return fig, f"Diagram saved as {pdf_path}"

    return fig, ""

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
